In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from pycaret.classification import *
import time

In [2]:
def recall_n(pred_p, true_l):
    n, cur_sum = sum(true_l), 0
    rank_pair = sorted(zip(pred_p,true_l),key=lambda x: x[0],reverse=True)
    for i, (_p, _l) in enumerate(rank_pair):
        cur_sum += _l
        if cur_sum==n:
            return n/(i+1)
        
def recall_n_1(pred_p, true_l):
    n, cur_sum = sum(true_l)-1, 0
    rank_pair = sorted(zip(pred_p,true_l),key=lambda x: x[0],reverse=True)
    for i, (_p, _l) in enumerate(rank_pair):
        cur_sum += _l
        if cur_sum==n:
            return n/(i+1)

In [5]:
seed_list=[42,142,242,342,442,1000,2000,3000,4000,5000,6000,7000,8000,9000,10000,
           13,113,213,313,413,513,613,713,813,913,1013,1113,1213,1313,1413,
           79,179,279,379,479,579,679,779,879,979,1079,1179,1279,1379,1479,
           721,1721,2721,3721,4721
          ]
train_csv_list = ['../csv/df_train_290.csv','csv/df_train_280.csv']
model_list = ['stacker2']

In [12]:
seed_list=[42]
train_csv_list = ['df_train_290.csv']
model_list = ['stacker2']

In [13]:
df_ = pd.DataFrame(columns=['csv','seed','cols','lgb','xgb','xgb2','blender','blender2','stacker','stacker_n','stacker2'])

In [10]:
df_test = pd.read_csv('../csv/df_test.csv')
y=[355091,355152,355724,363033,358453,359668,363320,363896,356602,354939,361617]
df_test['sar_flag']=np.where(df_test.alert_key.isin(y),1,0)

In [15]:
start = time.time()
for seed in seed_list:
    print(str(seed) +" seed 執行時間：%f 秒" % ( time.time() - start))
    for csv in train_csv_list:
        print(csv+" 執行時間：%f 秒" % ( time.time() - start))
        df_train = pd.read_csv('../csv/'+csv)
        cols=0
        lgb_score=0
        xgb_score=0
        xgb2_score=0
        blender_score=0
        blender2_score=0
        stacker_score=0
        stacker_score_n=0
        stacker2_score=0
        s = setup(data = df_train, target = 'sar_flag', session_id=seed, train_size=0.8,
                   ignore_features=['alert_key','cust_id','date','maxdate','ATM_sum',
                          'date_count','tx_amt_max_3','tx_amt_mean_3','tx_amt_std_3'])
        X_train = get_config('X_train')
        cols = len(X_train.columns)
        
        lgb=create_model('lightgbm')
        xgb=create_model('xgboost')
        xgb2=create_model('xgboost', max_depth=6,  learning_rate=0.2, fold=10, tree_method='gpu_hist', gpu_id=0, sort='Recall')
        blender=blend_models(estimator_list = [lgb,xgb], method = 'soft',fold=5)
        
        for model_name in model_list:
            print(model_name+ " 執行時間：%f 秒" % ( time.time() - start))
            if model_name=='lgb':
                new_model=lgb
            elif model_name=='xgb':                
                new_model= xgb
            elif model_name=='xgb2':                
                new_model=xgb2 
            elif model_name=='blender':                
                new_model = blender
            elif model_name=='blender2':                
                new_model = blender2
            elif model_name=='stacker':              
                new_model = stack_models([xgb,lgb,blender],meta_model=xgb,fold=5)
            elif model_name=='stacker2':
                new_model = stack_models([xgb2,lgb,blender],meta_model=xgb2,fold=5)                
        
            print(model_name)
            pred_holdout3 = predict_model(new_model, data=df_test) #boosted_lgb #bagged_lgb
            pred_holdout3['proba']=np.where(pred_holdout3.prediction_label==1,pred_holdout3.prediction_score, 1-pred_holdout3.prediction_score)
            pred_holdout3=pd.concat([df_test[['cust_id','alert_key','date','maxdate']],pred_holdout3],axis=1)
            pred_holdout3[['alert_key','proba']].to_csv('../output/'+str(seed)+'_'+model_name+'_'+csv, index=False)
            
            score=recall_n_1(pred_holdout3.proba.to_list(), pred_holdout3.sar_flag.to_list())
            score_n=recall_n(pred_holdout3.proba.to_list(), pred_holdout3.sar_flag.to_list())
            
            if model_name=='lgb':
                lgb_score=score 
            elif model_name=='xgb':                
                xgb_score=score 
            elif model_name=='xgb2':                
                xgb2_score=score                 
            elif model_name=='blender':                
                blender_score = score
            elif model_name=='blender2':                
                blender2_score = score                
            elif model_name=='stacker':                  
                stacker_score = score
                stacker_score_n = score_n
            elif model_name=='stacker2':                  
                stacker2_score = score
                
        df_=df_.append({'csv' : csv , 'seed' : seed, 'cols' : cols,
                       'lgb':lgb_score,'xgb':xgb_score,'xgb2':xgb2_score,
                        'blender':blender_score,'blender2':blender2_score,
                        'stacker':stacker_score,'stacker_n': stacker_score_n,
                        'stacker2':stacker2_score} , ignore_index=True)
print("執行時間：%f 秒" % ( time.time() - start))

42 seed 執行時間：0.000595 秒
df_train_290.csv 執行時間：0.000663 秒


,Description,Value
0,Session id,42
1,Target,sar_flag
2,Target type,Binary
3,Original data shape,"(18821, 44)"
4,Transformed data shape,"(18821, 44)"
5,Transformed train set shape,"(15056, 44)"
6,Transformed test set shape,"(3765, 44)"
7,Ignore features,9
8,Numeric features,43
9,Rows with missing values,100.0%


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9887,0.9519,0.0556,1.0000,0.1053,0.1041,0.2344
1,0.9834,0.9354,0.0000,0.0000,0.0000,-0.0067,-0.0075
2,0.9867,0.9619,0.2778,0.4167,0.3333,0.3269,0.3338
3,0.9867,0.9264,0.1667,0.3750,0.2308,0.2251,0.2441
4,0.9880,0.9358,0.1111,0.5000,0.1818,0.1782,0.2318
5,0.9874,0.9519,0.0556,0.3333,0.0952,0.0921,0.1321
6,0.9894,0.9548,0.2353,0.5714,0.3333,0.3289,0.3623
7,0.9880,0.9525,0.1111,0.5000,0.1818,0.1782,0.2318
8,0.9887,0.9629,0.1111,0.6667,0.1905,0.1877,0.2692


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9874,0.9473,0.0556,0.3333,0.0952,0.0921,0.1321
1,0.9847,0.9432,0.0556,0.1429,0.0800,0.0738,0.0823
2,0.9874,0.9609,0.2222,0.4444,0.2963,0.2906,0.3086
3,0.9867,0.9230,0.1111,0.3333,0.1667,0.1617,0.1870
4,0.9854,0.9373,0.1111,0.2500,0.1538,0.1476,0.1601
5,0.9874,0.9702,0.0000,0.0000,0.0000,-0.0013,-0.0028
6,0.9874,0.9587,0.0588,0.2500,0.0952,0.0913,0.1166
7,0.9880,0.9573,0.1111,0.5000,0.1818,0.1782,0.2318
8,0.9874,0.9545,0.1111,0.4000,0.1739,0.1696,0.2061


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9854,0.9564,0.0556,0.1667,0.0833,0.0778,0.0900
1,0.9841,0.9553,0.0556,0.1250,0.0769,0.0701,0.0760
2,0.9880,0.9694,0.1111,0.5000,0.1818,0.1782,0.2318
3,0.9894,0.9463,0.2222,0.6667,0.3333,0.3293,0.3810
4,0.9867,0.9486,0.0556,0.2500,0.0909,0.0869,0.1130
5,0.9887,0.9699,0.1111,0.6667,0.1905,0.1877,0.2692
6,0.9874,0.9600,0.1765,0.3750,0.2400,0.2345,0.2516
7,0.9887,0.9668,0.1111,0.6667,0.1905,0.1877,0.2692
8,0.9867,0.9641,0.1111,0.3333,0.1667,0.1617,0.1870


...

stacker2 執行時間：27.424587 秒


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9877,0.9535,0.0833,0.4286,0.1395,0.1362,0.1850
1,0.9877,0.9618,0.1429,0.4167,0.2128,0.2081,0.2390
2,0.9870,0.9671,0.0833,0.3333,0.1333,0.1292,0.1619
3,0.9880,0.9497,0.0833,0.5000,0.1429,0.1399,0.2006
4,0.9874,0.9490,0.1111,0.4000,0.1739,0.1696,0.2061
Mean,0.9876,0.9562,0.1008,0.4157,0.1605,0.1566,0.1985
Std,0.0003,0.0071,0.0236,0.0535,0.0297,0.0292,0.0254


stacker2


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Stacking Classifier,0.9930,0.9171,0.1818,0.1000,0.1290,0.1258,0.1315


執行時間：57.673781 秒


In [16]:
file_list=[]
for i, row in df_.iterrows():
    if row['stacker2']>=0.025:
        file_list.append( f"{row['seed']}_stacker2_{row['csv']}")
    if row['blender']>=0.025:
        file_list.append(f"{row['seed']}_blender_{row['csv']}")    

In [17]:
df_.to_csv('../output/final_output.csv', index=False)